In [1]:
#https://medium.com/@viniciusqroz/using-keras-tuner-to-find-the-best-parameters-for-your-neural-network-model-2dc02e0a1203

In [2]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.8 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

<ipython-input-3-faf0bbded06d>:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [4]:
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first dense layer
    # Choose an optimal value between 32-512
    hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units1, activation='relu'))

    # Tune the number of units in the second dense layer
    # Choose an optimal value between 32-512
    hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units2, activation='relu'))

    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])

    model.add(layers.Dense(10, activation='softmax'))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [6]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

In [7]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

Trial 3 Complete [00h 01m 51s]
val_accuracy: 0.7010999917984009

Best val_accuracy So Far: 0.8024666508038839
Total elapsed time: 00h 08m 27s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
288               |288               |units1
384               |64                |units2
0.0001            |0.0001            |learning_rate

Epoch 1/5
1875/1875 [==============================] - 16s 8ms/step - loss: 4.8449 - accuracy: 0.7622 - val_loss: 2.1579 - val_accuracy: 0.8032
Epoch 2/5
1875/1875 [==============================] - 14s 8ms/step - loss: 1.5966 - accuracy: 0.8127 - val_loss: 1.4514 - val_accuracy: 0.7981
Epoch 3/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.9830 - accuracy: 0.8273 - val_loss: 1.0995 - val_accuracy: 0.8016
Epoch 4/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.7040 - accuracy: 0.8391 - val_loss: 0.8497 - val_accuracy: 0.8113
Epoch 5/5
1875/1875 [==============================] - 15s 

KeyboardInterrupt: 

In [ ]:
best_models = tuner.get_best_models(num_models=1)

In [ ]:
test_loss, test_acc = best_models[0].evaluate(x_test, y_test)
print('Test accuracy:', test_acc)